In [8]:
import numpy as np
import pandas as pd
from scipy import stats
from random import uniform,randint
import uuid

# Дельта метод

**Дельта метод** - это метод обсчета результатов АБ теста по исследованию метрик отношения. 

**Метрика отношения** - это когда у каждого пользователя в выборках есть множество каких-то значений и мы получаем новую метрику, усредняя их.
Например, у каждого юзера было множество покупок за период с разной ценой. Мы хотим узнать средний чек каждого юзера - это метрика отношения. 

**Суть метода:**
    
1. Каждому юзеру аггрегируем кол-во (Y) и сумму (X) его значений. Хотим исследовать метрику отношения X/Y.

2. Считаем дисперсии в каждой выборке по формуле:
    
$$
V\left(\frac{X}{Y}\right) \approx \frac{1}{|A|} \left( \frac{1}{\mu_Y^2} V(X) - \frac{2 \mu_X}{\mu_Y^3} \text{cov}(X, Y) + \frac{\mu_X^2}{\mu_Y^4} V(Y) \right)
$$

**X, Y** - столбцы с значениями числителя и знаменателя

**VX** - дисперсия значений числителей (напр. значений сумм покупок по юзерам).

**VY** - дисперсия значений знаменателей (напр. значений количеств покупок по юзерам)

|A| - размер выборки

mu_Х - среднее значений числителей 

mu_Y - среднее значений знаменателей



3. Считаем t-статистику по формуле:
$$
t = \frac{\mathcal{R}_B - \mathcal{R}_A}{\sqrt{V(\mathcal{R}_A) + V(\mathcal{R}_B)}} \sim N(0,1)
$$

**R** - полученная метрика отношения по всей выборке, то есть сумму всех числителей делим на всю сумму знаменателей (напр.: сумму всех чеков делим на сумму количеств всех покупок)

**VR** - дисперсия, вычисляемая по вышеуказанной формуле.

## Пример (средний чек):

In [131]:
# получаем пример датафрейм с набором юзеров и их чеков:

# тестовая выборка:
users = [str(uuid.uuid4()) for i in range(100)]
bills = []

for user in users:
    bills_of_user = np.random.uniform(2, 11, randint(1,50)) # рандомное кол-во покупок 1..50 каждая ценой 2...11 (выше) 
    bills.append(bills_of_user)    

data_test = pd.DataFrame({'user_id':users, 'bill':bills}).explode('bill')

# контрольная выборка:
users = [str(uuid.uuid4()) for i in range(1000)]
bills = []

for user in users:
    bills_of_user = np.random.uniform(1, 10, randint(1,50)) # рандомное кол-во покупок 1..50 каждая ценой 1...10 
    bills.append(bills_of_user)    

data_control = pd.DataFrame({'user_id':users, 'bill':bills}).explode('bill')

data_control.head()

,user_id,bill
0,af741a7a-3f63-48db-9d2b-88535764a445,5.145656
0,af741a7a-3f63-48db-9d2b-88535764a445,2.438346
0,af741a7a-3f63-48db-9d2b-88535764a445,1.403702
0,af741a7a-3f63-48db-9d2b-88535764a445,1.330163
0,af741a7a-3f63-48db-9d2b-88535764a445,3.563273


In [132]:
# предподготавливаем данные:
preprocessed_test = data_test.groupby(['user_id'], as_index=False).agg({"bill":['sum', 'count']})
preprocessed_test.columns = ['user_id', 'bill_sum', 'bill_count']
preprocessed_test['bill_sum'] = preprocessed_test['bill_sum'].astype(float)

preprocessed_control = data_control.groupby(['user_id'], as_index=False).agg({"bill":['sum', 'count']})
preprocessed_control.columns = ['user_id', 'bill_sum', 'bill_count']
preprocessed_control['bill_sum'] = preprocessed_control['bill_sum'].astype(float)

print(f'Размер теста: {len(preprocessed_test)}')
print(f'Размер контроля: {len(preprocessed_control)}')

preprocessed_control.head()

Размер теста: 100
Размер контроля: 1000


,user_id,bill_sum,bill_count
0,000f8e7a-1dc3-40f0-a9db-054e45495911,199.241234,40
1,008e8f30-eed5-461b-a379-87e49be8f1af,155.652318,28
2,00ab0394-4bdb-4259-8002-c9d5b1a62793,4.991587,2
3,010faf8f-cd27-4693-bd4d-e22a7c56d2e4,193.654742,37
4,0111d923-d4aa-42e3-93b6-7d52ac93e5c8,120.137586,18


In [133]:
import numpy as np

# Средние значения
avg_x = preprocessed_test['bill_sum'].mean()
avg_y = preprocessed_test['bill_count'].mean()

# Дисперсии
var_x = preprocessed_test['bill_sum'].var()
var_y = preprocessed_test['bill_count'].var()

# Размер выборки
n = len(preprocessed_test)

# Ковариация
cov = np.cov(preprocessed_test['bill_sum'], preprocessed_test['bill_count'])[0, 1]

# Вычисление дисперсии отношения
(var_x / (avg_y ** 2) - 2 * avg_x * cov / (avg_y ** 3) + (avg_x ** 2) * var_y / (avg_y ** 4)) / n


0.0029484946960156755

In [134]:
var_test

0.006877192574472133

In [135]:
# считаем дисперсии:

# тест:
avg_x = preprocessed_test['bill_sum'].mean()
avg_y = preprocessed_test['bill_count'].mean()
var_x = preprocessed_test['bill_sum'].var()
var_y = preprocessed_test['bill_count'].var()
n = len(preprocessed_test)
cov = np.cov(preprocessed_test['bill_sum'], preprocessed_test['bill_count'])[0, 1]

var_test = (var_x/(avg_y**2) - 2*avg_x*cov/(avg_y**3) + (avg_x**2)*var_y/(avg_y**4)) / n
# контроль:
avg_x = preprocessed_control['bill_sum'].mean()
avg_y = preprocessed_control['bill_count'].mean()
var_x = preprocessed_control['bill_sum'].var()
var_y = preprocessed_control['bill_count'].var()
n = len(preprocessed_control)
cov = np.cov(preprocessed_control['bill_sum'], preprocessed_control['bill_count'])[0, 1]

var_control = (var_x/(avg_y**2) - 2*avg_x*cov/(avg_y**3) + ((avg_x**2)*var_y)/(avg_y**4)) / n

print(f'Дисперсия теста: {var_test:.5f}')
print(f'Дисперсия контроля: {var_control:.5f}')

Дисперсия теста: 0.00295
Дисперсия контроля: 0.00025


In [136]:
# считаем t-статистику:
r_test = preprocessed_test['bill_sum'].sum() / preprocessed_test['bill_count'].sum()
r_control = preprocessed_control['bill_sum'].sum() / preprocessed_control['bill_count'].sum()

t_statistic = (r_test-r_control) / np.sqrt(var_test+var_control)

In [138]:
# считаем p-value:
pvalue = 2 * stats.norm.cdf(-abs(t_statistic))
pvalue

2.132527261671235e-63

Формулу с MDE в этом случае надо использовать подставляя вместо станд. отклонения - корень из полученных дисперсий. 
MDE - мин. эффект в рассматриваемых единицах (напр. 10 руб.)

# Линеаризация: